In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
feature_x_signature = pd.DataFrame(index=w.index)

for w_or_h, element_x_factor, element_x_signature in (("w", w, feature_x_signature),):

    for factor_name, factor_values in element_x_factor.items():

        element_x_signature.loc[
            ccal.select_series_indices(
                factor_values,
                ">",
                standard_deviation=1,
                title="MF K={} {} {} Signature".format(
                    SETTING["mf_k"], w_or_h.title(), factor_name
                ),
                xaxis={"title": "Rank"},
                yaxis={"title": "Factor Value"},
            ),
            "{} Signature".format(factor_name),
        ] = 1

feature_x_signature.fillna(0, inplace=True)

In [ ]:
gps_map = ccal.read_gps_map(PATH["gps_map.pickle.gz"])

if gps_map.w_element_label is None:

    gps_map_directory_path = PATH["w|gps_map/"]

else:

    gps_map_directory_path = PATH["w|hcc|gps_map/"]

for signature, feature_is_factor in feature_x_signature.items():

    gps_map.plot(
        "w",
        annotation_x_element=feature_is_factor.to_frame().T,
        annotation_types=("binary",),
        element_marker_size=SETTING["gps_map_w_element_marker_size"],
        title=signature,
        html_file_path=os.path.join(gps_map_directory_path, signature),
    )

In [ ]:
if os.path.isfile(PATH["w|cluster_x_element.tsv"]):

    w_element_cluster = pd.read_csv(
        PATH["w|cluster_x_element.tsv"], sep="\t", index_col=0
    ).apply(lambda column: column.argmax())

    w_element_cluster.name = "Cluster"

    w_element_cluster.index.name = w.index.name

else:

    w_element_cluster = None

if os.path.isfile(PATH["h|cluster_x_element.tsv"]):

    h_element_cluster = pd.read_csv(
        PATH["h|cluster_x_element.tsv"], sep="\t", index_col=0
    ).apply(lambda column: column.argmax())

    h_element_cluster.name = "Cluster"

    h_element_cluster.index.name = h.columns.name

else:

    h_element_cluster = None

In [ ]:
feature_summary = pd.concat((w, feature_x_signature, w_element_cluster), axis=1)

feature_summary.to_csv(
    os.path.join(PATH["summary/"], "feature_x_summary_information.tsv"), sep="\t"
),

feature_summary

In [ ]:
sample_summary = pd.concat((h.T, h_element_cluster), axis=1)

sample_summary.to_csv(
    os.path.join(PATH["summary/"], "sample_x_summary_information.tsv"), sep="\t"
),

sample_summary